In [1]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Using numpy backend.
Using numpy backend.
Using numpy backend.
Using numpy backend.


Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'

Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'

Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.
Using pytorch backend.
Using pytorch backend.


In [ ]:
tag = [72, 71, 84, 54, 57, 65, 64]
Y = np.zeros(467)
print(Y.shape)
Y[0 : tag[0]] = 0
Y[tag[0] : tag[0]+tag[1]] = 1
Y[tag[0]+tag[1] : tag[0]+tag[1]+tag[2]] = 2
Y[tag[0]+tag[1]+tag[2] : tag[0]+tag[1]+tag[2]+tag[3]] = 3
Y[tag[0]+tag[1]+tag[2]+tag[3] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]] = 4
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]] = 5
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]+tag[6]] = 6
Y = Y.tolist()

In [ ]:
inputs = []
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    if a.shape[1] > 16:
        pca = PCA(n_components=16)
        a = pca.fit_transform(a)
    else:
        b = np.zeros((467,16 - a.shape[1]))
        a = np.concatenate((a, b), axis=1)
    a = a.transpose(1,0)
    inputs.append((torch.from_numpy(a)))

In [ ]:
use = torch.stack(inputs)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation

In [ ]:
inputs = []
for i in range(0,467):
    inputs.append((use[i,:,:] , Y[i]))
inputs[0][0].shape

In [ ]:
import random
random.shuffle(inputs)
train_inputs = inputs[0 : int(len(inputs)*0.7)]
test_inputs = inputs[int(len(inputs)*0.7) : -1]
train_inputs_x = []
train_inputs_y = []
test_inputs_x = []
test_inputs_y = []
for res in train_inputs:
    train_inputs_x.append(res[0])
    train_inputs_y.append(res[1])
for res in test_inputs:
    test_inputs_x.append(res[0])
    test_inputs_y.append(res[1])

test_inputs_x = torch.stack(test_inputs_x)
test_inputs_y = torch.tensor(test_inputs_y)
train_inputs_x = torch.stack(train_inputs_x)
train_inputs_y = torch.tensor(train_inputs_y)
train_inputs_x.shape

In [ ]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 467,       # rnn input
            hidden_size = 128,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=128,out_features=7)

    def forward(self,x):
        output,(h_n,c_n)=self.rnn(x.float())
        output_in_last_timestep=h_n[-1,:,:]
        x=self.out(output_in_last_timestep)
        return x

# 向量构建：0.34285714285714286

In [ ]:
# 创建数据加载器
loader = DataLoader(
    dataset = train_inputs, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = RNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(300): 
    for step, (batch_x, batch_y) in enumerate(loader):
        pred = net(batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch == 50:
            optimizer.lr = 0.001
        if step % 50 == 0: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(test_inputs_x)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == test_inputs_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")